<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Face-Extraction" data-toc-modified-id="Face-Extraction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Face Extraction</a></span><ul class="toc-item"><li><span><a href="#TMP-Landmarks-To-Image" data-toc-modified-id="TMP-Landmarks-To-Image-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>TMP Landmarks To Image</a></span></li></ul></li><li><span><a href="#Face-Alignment" data-toc-modified-id="Face-Alignment-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Face Alignment</a></span><ul class="toc-item"><li><span><a href="#Landmark-Detection" data-toc-modified-id="Landmark-Detection-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Landmark Detection</a></span></li><li><span><a href="#Find-Convex-Hull" data-toc-modified-id="Find-Convex-Hull-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Find Convex Hull</a></span></li><li><span><a href="#Delaunay-Triangulation" data-toc-modified-id="Delaunay-Triangulation-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Delaunay Triangulation</a></span><ul class="toc-item"><li><span><a href="#Plot-Triangulation" data-toc-modified-id="Plot-Triangulation-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Plot Triangulation</a></span></li></ul></li><li><span><a href="#Affine-warp-triangles" data-toc-modified-id="Affine-warp-triangles-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Affine warp triangles</a></span></li></ul></li><li><span><a href="#Face-Blending" data-toc-modified-id="Face-Blending-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Face Blending</a></span></li><li><span><a href="#Face-Generation" data-toc-modified-id="Face-Generation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Face Generation</a></span><ul class="toc-item"><li><span><a href="#Autoencoder-Generation" data-toc-modified-id="Autoencoder-Generation-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Autoencoder Generation</a></span></li></ul></li><li><span><a href="#Hyperparams-Eval" data-toc-modified-id="Hyperparams-Eval-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Hyperparams Eval</a></span></li></ul></div>

# Intro
Notebook exploring face related operations in Python, e.g. face extraction, alignment, blending, swapping.

Resources:
* [Face Swap using OpenCV](https://www.learnopencv.com/face-swap-using-opencv-c-python/)
* [How to install dlib](https://www.pyimagesearch.com/2017/03/27/how-to-install-dlib/)
* [Detect eyes, nose, lips, and jaw with dlib, OpenCV, and Python](https://www.pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
import sys
import yaml

import dlib
import cv2
import tensorflow as tf
from IPython import display
from IPython.display import clear_output
from PIL import Image
from numpy.random import shuffle

from tqdm import tqdm

# Plotting
%matplotlib notebook
#%matplotlib inline

sns.set_context("paper")
sns.set_style("dark")

# face-swap repo
sys.path.append('../face_swap')

import FaceGenerator
from face_swap import faceswap_utils as utils
from face_swap import gan_utils, plot_utils
import FaceGenerator, FaceDetector
import autoencoder
import gan
from face_swap import CONFIG_PATH
from Face import Face
from deep_swap import swap_faces, Swapper
from face_swap.train import get_original_data

# data-science-learning utils
from utils import image_processing

%load_ext autoreload
%autoreload 2

In [ ]:
data_folder = Path.home() / "Documents/datasets/"
models_folder = Path.home() / "Documents/models/"

# Load Data

In [ ]:
def load_img(path: Path, convert_BGR2RGB=False):
    img = cv2.imread(str(path))
    if convert_BGR2RGB:
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        return img

In [ ]:
# Load two random celeba faces
rand_celeba_count = np.random.randint(0, 9, 3)
from_face_img = load_img(data_folder / "img_align_celeba" / "000{}{}{}.jpg".format(*rand_celeba_count),
                             True)
rand_celeba_count = np.random.randint(0, 9, 3)
to_face_img = load_img(data_folder / "img_align_celeba" / "000{}{}{}.jpg".format(*rand_celeba_count),
                             True)

In [ ]:
plt.imshow(from_face_img)
plt.show()
plt.imshow(to_face_img)
plt.show()

# Face Extraction

In [ ]:
# Init face detector by passing local config
with open(CONFIG_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

#tf.reset_default_graph()
cfg['extract']['align'] = False
cfg['extract']['masked'] = False

face_detector = FaceDetector.FaceDetector(cfg)

In [ ]:
faces = face_detector.detect_faces(from_face_img)

In [ ]:
faces[0].landmarks

## TMP Landmarks To Image

In [ ]:
from PIL import Image, ImageDraw
def draw_landmarks(face, height, width, out_path):
    img = Image.new('L', (width, height), 255)
    draw = ImageDraw.Draw(img)
    # Jawline
    draw.line(face[0:17])

    # Right Brow
    draw.line(face[17:22])

    # Left Brow
    draw.line(face[22:27])

    # Nose Line
    draw.line(face[27:31])

    # Nose Base
    draw.line(face[31:36])

    # Right Eye
    draw.polygon(face[36:42])

    # Left Eye
    draw.polygon(face[42:48])

    # Outside Mouth
    draw.polygon(face[48:60])

    # Inside Mouth
    draw.polygon(face[60:68])

    img.save(out_path)
    return img

# Face Alignment

## Landmark Detection

In [ ]:
# get face boundary points and containing rectangles
# for both faces
face_boundary_from, rect_from = landmark_detector.get_contour(from_face)
face_boundary_to, rect_to = landmark_detector.get_contour(to_face)

## Find Convex Hull
Get convex hull indexes only of target face, and obtain hull points for both faces using such indexes.

In [ ]:
#hull_idx_from = cv2.convexHull(face_boundary_from, returnPoints = False)
hull_idx_to = cv2.convexHull(face_boundary_to, returnPoints = False)

In [ ]:
#??Do not use directly this cause you might lose correspondence
#between number of points
#hull_from = cv2.convexHull(face_boundary_from, returnPoints = True)
#hull_to = cv2.convexHull(face_boundary_to, returnPoints = True)

In [ ]:
hull_from = np.array([face_boundary_from[hull_idx] for hull_idx in hull_idx_to])
hull_to = np.array([face_boundary_to[hull_idx] for hull_idx in hull_idx_to])

## Delaunay Triangulation

In [ ]:
#triangles_from_idxs = utils.get_triangles_indexes(from_face, rect_from, hull_idx_to, face_boundary_from)
triangles_to_idxs = utils.get_triangles_indexes(to_face, rect_to, hull_idx_to, face_boundary_to)

In [ ]:
triangles_from = utils.clean_triangles((0, 0, from_face.shape[1], from_face.shape[0]), 
                                 utils.delaunay_triangulation(from_face, rect_from, hull_idx_from, face_boundary_from))
triangles_to = utils.clean_triangles((0, 0, to_face.shape[1], to_face.shape[0]), 
                               utils.delaunay_triangulation(to_face, rect_to, hull_idx_to, face_boundary_to))

In [ ]:
print(len(triangles_from))
print(len(triangles_to))

### Plot Triangulation

In [ ]:
import matplotlib.patches as patches

In [ ]:
# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(from_face)

for t in triangles_from:
    triangle = patches.Polygon(t, linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(triangle)
# Create a Rectangle patch
#rect = patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
#ax.add_patch(rect)
plt.show()

In [ ]:
# Create figure and axes
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(to_face)

for t in triangles_to:
    triangle = patches.Polygon(t, linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(triangle)
# Create a Rectangle patch
#rect = patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
#ax.add_patch(rect)
plt.show()

In [ ]:
np.array(triangles_to[0])

## Affine warp triangles 

In [ ]:
img_res = utils.image_affine_warp(hull_from,
                      hull_to,
                      triangles_to_idxs, 
                      from_face, 
                      to_face.copy())

In [ ]:
plt.imshow(img_res)
plt.show()

# Face Blending

In [ ]:
with open(CONFIG_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

#tf.reset_default_graph()
cfg['extract']['align'] = True
face_detector = FaceDetector.FaceDetector(cfg)
model_cfg = cfg['base_gan']['v0']
aut_a, aut_b, _, _ = gan.get_gan(model_cfg)
target_aut = aut_a if cfg.get('use_aut_a') else aut_b
face_generator = FaceGenerator.FaceGenerator(
        lambda seed_face, size: FaceGenerator.aue_generate_face(target_aut,
                                                                seed_face, size,
                                                                tanh_fix=True,
                                                                masked=False))

In [ ]:
faces = face_detector.detect_faces(cv2.cvtColor(to_face_img, cv2.COLOR_RGB2BGR))
face = faces[0]
face.landmarks = face_detector.get_landmarks(face)

In [ ]:
cfg['swap']['face_size'] = (64, 64)
swap_res = swap_faces(face, face_detector, cfg['swap'], face_generator)

In [ ]:
plt.imshow(cv2.cvtColor(swap_res, cv2.COLOR_BGR2RGB))
plt.show()

# Face Generation

In [ ]:
# generate random image
rnd_image = FaceGenerator.generate_random_img(None, (28, 28))
print(rnd_image.shape)

In [ ]:
plt.imshow(rnd_image)
plt.show()

In [ ]:
# random face transform
rnd_face = FaceGenerator.random_transform(from_face)
print(rnd_face.shape)

In [ ]:
plt.imshow(rnd_face)
plt.show()

In [ ]:
# random face warp
warped_face, target_face = FaceGenerator.random_warp(from_face_img)
print(warped_face.shape)

In [ ]:
plt.imshow(warped_face)
plt.show()

In [ ]:
plt.imshow(target_face)
plt.show()

## Autoencoder Generation

In [ ]:
aut_models_folder = models_folder / 'face_recognition'

In [ ]:
aut_A, aut_B = autoencoder.get_autoencoders(str(aut_models_folder / 'v12k'))

In [ ]:
tmp_face = Face(to_face_img, None)
tmp_face.face_img = tmp_face.img

In [ ]:
gen_face = FaceGenerator.aue_generate_face(aut_A, tmp_face, (64, 64))
print(gen_face.shape)

In [ ]:
cv2.imshow("", gen_face)
#plt.show()

In [ ]:
face_generator = FaceGenerator.FaceGenerator(lambda x, y: FaceGenerator.aue_generate_face(aut_A, x, y))

# Hyperparams Eval

In [ ]:
target_img = cv2.imread(str(data_folder / "face_swap"))

In [ ]:
# load config
with open(CONFIG_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
model_cfg = cfg['masked_gan']['v1']

#model_cfg['models_path'] = None
#model_cfg['img_shape'] = "(128, 128, 3)"
#model_cfg['discriminator_input_shape'] = "(128, 128, 6)"
#model_cfg['decoder_nb_deconv_blocks'] = 4

In [ ]:
tf.reset_default_graph()
face_detector = FaceDetector.FaceDetector(cfg)

In [ ]:
# load generator and related functions
gen_a, gen_b, _, _ = gan.get_gan(model_cfg, load_discriminators=True)
_, _, _, fun_generate_a, fun_mask_a, fun_abgr_a = gan_utils.cycle_variables_masked(gen_a)
gen_fun_a = lambda x: fun_abgr_a([np.expand_dims(x, 0)])[0][0]

In [ ]:
generator_a = FaceGenerator.FaceGenerator(
            lambda face_img: FaceGenerator.gan_masked_generate_face(gen_fun_a, face_img),
            input_size=(64, 64), config=cfg['swap'])

In [ ]:
swapper = Swapper(face_detector, generator_a, cfg['swap'])

In [ ]:
# blur_size
dest_folder = data_folder / "faceswap_experiments/hyperparams_test/blur_size"
for blur_size in np.arange(0, 20, 3):
    swapper.config['blur_size'] = blur_size
    results = swapper.swap(target_img)
    cv2.imwrite(str(dest_folder / "{}.png".format(blur_size)), results)

In [ ]:
# color_correct_blur_frac
dest_folder = data_folder / "faceswap_experiments/hyperparams_test/col_cor"
if not dest_folder.exists():
    dest_folder.mkdir()
for color_correct_blur_frac in np.linspace(0, 1, 10):
    swapper.config['color_correct_blur_frac'] = color_correct_blur_frac
    results = swapper.swap(target_img)
    cv2.imwrite(str(dest_folder / "{:.2f}.png".format(color_correct_blur_frac)), results)

In [ ]:
# mask_method
dest_folder = data_folder / "faceswap_experiments/hyperparams_test/mask_method"
if not dest_folder.exists():
    dest_folder.mkdir()
for mask_method in ['gen_mask', 'gen_mask_fix', 'face_mask', 'mix_mask']:
    swapper.config['mask_method'] = mask_method
    results = swapper.swap(target_img)
    cv2.imwrite(str(dest_folder / "{}.png".format(mask_method)), results)